## C S 329E HW 6

# KNN 


In [51]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
%matplotlib inline

# Starting off loading a training set
df_melb = pd.read_csv('melb_data_train.csv')
df_melb.head()

,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt
0,3,t,732000,26/07/2016,5.6,3101,1,1.0,904,110.0,1980.0
1,3,h,1001000,4/3/17,12.6,3020,1,5.0,879,NaN,NaN
2,2,u,605000,7/5/16,7.4,3185,1,1.0,722,NaN,1970.0
3,3,h,757500,17/06/2017,18.8,3170,2,1.0,145,NaN,NaN
4,4,h,721000,16/09/2017,17.9,3082,2,2.0,603,NaN,NaN


FIX A COLUMN TO BE NUMERIC 

In [52]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    
    #don't alter the original variable and make a copy
    date_value = d
    
    #if the date doesn't have a slash, then the first value is single digit
    if date_value[2] != '/':
        const = '0' + date_value
        date_value = const
    #if the month doesn't have a slash, then month single digit, adjust accordingly
    if date_value[5] != '/':
        month_value = date_value[:3] + '0' + date_value[3:]
        date_value = month_value
    #adjust the date to the year format
    if len(date_value) < 10:
        year_value = date_value[:6] + '20' + date_value[6:]
        date_value = year_value
        
    #return the date value
    return date_value
    

In [53]:
df_melb['Date'] = df_melb['Date'].apply( lambda x : standardize_date(x)) 
df_melb['unixtime'] = df_melb['Date'].apply( lambda x: time.mktime(time.strptime(x,"%d/%m/%Y")))
df_melb = df_melb.drop(columns="Date")
print("The min unixtime:", df_melb['unixtime'].min())
print("The max unixtime is:", df_melb['unixtime'].max())

The min unixtime: 1454565600.0
The max unixtime is: 1506142800.0


Use Imputation to fill in missing values


In [54]:
target_col = 'Type'

In [55]:
#create the dictionary
dict_imputation = dict()

#make sure that all the attributes are available for the columns and no nulls are found and if found replace
for col in df_melb.columns:
    if col != target_col:
        dict_imputation[col] = df_melb[col].mean(skipna=True)
        df_melb[col].fillna(value=dict_imputation[col],inplace=True)

In [56]:
dict_imputation

{'Rooms': 2.710769230769231,
 'Price': 941972.2953846154,
 'Distance': 10.206256410256412,
 'Postcode': 3110.873846153846,
 'Bathroom': 1.4543589743589744,
 'Car': 1.4938398357289528,
 'Landsize': 514.2184615384615,
 'BuildingArea': 131.379476861167,
 'YearBuilt': 1971.0204429301534,
 'unixtime': 1485054996.9230769}

In [57]:
df_melb.head()
df_melb.isnull().sum()

Rooms           0
Type            0
Price           0
Distance        0
Postcode        0
Bathroom        0
Car             0
Landsize        0
BuildingArea    0
YearBuilt       0
unixtime        0
dtype: int64

Normalize all the attributes to be between [0,1]


In [58]:
#create normalize dict
dict_normalize = dict()

#make the values to be ranges between 0 and 1 to graph
for col in df_melb.columns:
    if col != target_col:
        dict_normalize[col] = (df_melb[col].min(),df_melb[col].max())
        df_melb[col] = (df_melb[col] - dict_normalize[col][0])/(dict_normalize[col][1]-dict_normalize[col][0])

In [59]:
dict_normalize

{'Rooms': (1, 7),
 'Price': (210000, 5020000),
 'Distance': (0.7, 47.3),
 'Postcode': (3000, 3810),
 'Bathroom': (0, 5),
 'Car': (0.0, 8.0),
 'Landsize': (0, 41400),
 'BuildingArea': (0.0, 3558.0),
 'YearBuilt': (1850.0, 2016.0),
 'unixtime': (1454565600.0, 1506142800.0)}

In [60]:
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,0.333333,t,0.108524,0.105150,0.124691,0.2,0.125,0.021836,0.030916,0.783133,0.289733
1,0.333333,h,0.164449,0.255365,0.024691,0.2,0.625,0.021232,0.036925,0.729039,0.660013
2,0.166667,u,0.082121,0.143777,0.228395,0.2,0.125,0.017440,0.036925,0.722892,0.155720
3,0.333333,h,0.113825,0.388412,0.209877,0.4,0.125,0.003502,0.036925,0.729039,0.835834
4,0.500000,h,0.106237,0.369099,0.101235,0.4,0.250,0.014565,0.036925,0.729039,0.988274


Load in the Test data and prep it for classification


In [61]:
#download the test set
df_test = pd.read_csv("melb_data_test.csv")

In [62]:
#translate to seconds after epoch and unix time
df_test['Date'] = df_test['Date'].apply( lambda x : standardize_date(x)) 
df_test['unixtime'] = df_test['Date'].apply( lambda x: time.mktime(time.strptime(x, "%d/%m/%Y")))
df_test = df_test.drop(columns="Date")

print("The min unixtime:", df_test['unixtime'].min())
print("The max unixtime is:", df_test['unixtime'].max())

The min unixtime: 1454565600.0
The max unixtime is: 1506142800.0


In [29]:
#impute with values in the nulls

for col in df_test.columns:
    # your code here
    if col != target_col:
        dict_imputation[col] = df_test[col].mean(skipna=True)
        df_test[col].fillna(value=dict_imputation[col],inplace=True)

df_test.head()        

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,t,790000,11.2,3046,2,1.0,208,127.000000,2010.000000,1.497676e+09
1,3,h,1355000,8.8,3072,1,2.0,916,149.084074,1967.384615,1.476508e+09
2,5,h,2810000,6.3,3143,2,2.0,617,149.084074,1967.384615,1.472360e+09
3,3,h,850000,10.5,3034,1,1.0,593,118.000000,1970.000000,1.472879e+09
4,3,h,810000,38.0,3199,1,2.0,835,118.000000,1960.000000,1.499490e+09


In [30]:
#normalize with values between 0 and 1 (find the z scores basically)
dict_normalize = dict()
for col in df_test.columns:
    if col != target_col:
        dict_normalize[col] = (df_test[col].min(),df_test[col].max())
        df_test[col] = (df_test[col] - dict_normalize[col][0])/(dict_normalize[col][1]-dict_normalize[col][0])
        
df_test.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,0.166667,t,0.183188,0.263736,0.083732,0.5,0.166667,0.041418,0.246521,0.967742,0.835834
1,0.333333,h,0.407216,0.197802,0.145933,0.0,0.333333,0.182397,0.290426,0.624069,0.425421
2,0.666667,h,0.984140,0.129121,0.315789,0.5,0.333333,0.122859,0.290426,0.624069,0.345013
3,0.333333,h,0.206979,0.244505,0.055024,0.0,0.166667,0.118080,0.228628,0.645161,0.355064
4,0.333333,h,0.191118,1.000000,0.449761,0.0,0.333333,0.166268,0.228628,0.564516,0.871013


Write the kNN classifier function


In [63]:
#import neighbors
from sklearn.neighbors import KNeighborsClassifier

#find the nearest neighbors
def knn_class(df_train, k, target_col, observation ):
    df_train['distance'] = df_train.apply( lambda x : np.linalg.norm( x.drop(index=target_col) - observation), axis=1 )
    df_train = df_train.sort_values(by='distance').reset_index(drop=True)
    df_k = df_train[0:k]
    return df_k[target_col].value_counts().index[0]

Compute the accuracy using different k values

{1,3,13,25,50,100}

In [37]:
poss_k = [1,3,13,25,50,100] # possible k's
acc_k = [0,0,0,0,0,0] # the accuracy at each value of k

acc_indx = 0

#find the possible k's 
for k in poss_k:

    predictions = df_test.apply( lambda x : knn_class(df_melb.copy(), k, target_col, x.drop(index=target_col)), axis=1)
    correct = np.sum(predictions == df_test[target_col])
    acc_k[acc_indx] = correct/len(predictions)
    print("k = ", k , "Acc=", acc_k[acc_indx])
    acc_indx = acc_indx + 1


KeyboardInterrupt: 

In [ ]:
#plot and find the highest accuracy
plt.plot(poss_k,acc_k,'g-*')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('Accuracy on Test Set for Varied k')
plt.grid('on')
plt.show()

I would choose $k = 13 $ because it is the most accurate.